In [2]:
import theano
import theano.tensor as T
import numpy as np
import scipy.io as sio
import sys

In [3]:
sys.path.append('SanDeepLearn/')

In [4]:
path_to_data = '/home/supriya/IncrementalMT/sp2016.11-731/hw1/data/dev-test-train.de-en'

In [5]:
lines = [line.strip().split('|||') for line in open(path_to_data, 'r')]

In [9]:
german_lines, english_lines = [x[0].strip() for x in lines], [x[1].strip() for x in lines]

In [11]:
german_vocab = {}
vocab_ind = 0
for line in german_lines:
    line = line.split()
    for word in line:
        if word not in german_vocab:
            german_vocab[word] = vocab_ind
            vocab_ind += 1

In [18]:
english_vocab = {}
vocab_ind = 0
for line in english_lines:
    line = line.split()
    for word in line:
        if word not in english_vocab:
            english_vocab[word] = vocab_ind
            vocab_ind += 1